### Group Members:

Aishwarya Rajeev (ar73945) <br>
Carlee Allen (cea2475) <br>
Juhi Patel (jpp2464) <br>
Snehal Naravane (sn27429) <br>
Tanushree Balaji (tb33857) <br>

### Task A - Scraping - Shared as a separate colab file. The results of the scraping are the files that are used below.

In [1]:
import pandas as pd
a1 = pd.read_csv('assignment2_final_data.csv')
a2 = pd.read_csv('assignment2_final_data_new.csv')
a3 = pd.read_csv('assignment2_unstructured_fame.csv')

In [2]:
a = a1.append(a2)
a_df = a.append(a3)

df = a_df.drop(columns=['Unnamed: 0'])
comments = df.drop_duplicates().reset_index().drop(columns=['index'])

/var/folders/_j/hp54sf295b9bd0bczykcmwvm0000gn/T/ipykernel_27256/1259342665.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a = a1.append(a2)
/var/folders/_j/hp54sf295b9bd0bczykcmwvm0000gn/T/ipykernel_27256/1259342665.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_df = a.append(a3)


In [3]:
comments.to_csv('assignment2_collated_data.csv')

### TASK B - Top Attributes

In [4]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import webtext
from nltk.probability import FreqDist
 
nltk.download('webtext')

[nltk_data] Downloading package webtext to
[nltk_data]     /Users/tanushreedevi/nltk_data...
[nltk_data]   Package webtext is already up-to-date!


True

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

import pandas as pd
text_file = comments['product_review']
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

term_freq = pd.DataFrame(columns = ['word','count']) 

for i in range(len(text_file)):
    text = str(text_file[i])
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = str(no_punct) + str(char)

    tokens = [t for t in no_punct.split()]
    filtered_sentence = [w.lower() for w in tokens if not w.lower() in stop_words]
    #filtered_sentence_df = pd.DataFrame(filtered_sentence)
                
    counts = {}
    for words in filtered_sentence:
        if words in counts:
            counts[words] += 0
        else:
            counts[words] = 1

    counts_df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
    if len(counts_df)>0:
        counts_df.columns = ['word','count']
        term_freq = pd.concat([term_freq, counts_df], ignore_index=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tanushreedevi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
freq_nostop = pd.DataFrame(term_freq.groupby('word').sum())
freq_nostop = freq_nostop.sort_values(by='count',ascending=False).reset_index()
freq_nostop.to_csv('assignment2_termfreq_data.csv')
freq_nostop.head(10)

,word,count
0,head,3538
1,taste,2699
2,beer,2522
3,carbonation,1837
4,pours,1824
5,good,1690
6,dark,1685
7,sweet,1643
8,aroma,1598
9,lacing,1596


#### Top 4 attributes - head, taste, carbonation, pours

### TASK C - Cosine Similarity 

In [7]:
top_attributes = pd.read_csv('top_attributes.csv') ### File containing top 4 attributes identified from Task B
list_attributes = np.array(top_attributes['attribute_name'])
list_attributes_text = " ".join(list_attributes)
text_file = comments['product_review']

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
similarity = pd.DataFrame(columns = ['product_name', 'product_review', 'similarity_score'])

for i in range(len(text_file)):
    prod_name = comments['product_name'][i]
    text = str(text_file[i])
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = str(no_punct) + str(char)
            
    tokens = [t for t in no_punct.split()]
    filtered_sentence = [w.lower() for w in tokens if not w.lower() in stop_words]
    filtered_sentence_fnl = " ".join(filtered_sentence)
    
    documents = [filtered_sentence_fnl,list_attributes_text]

    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['doc1', 'features'])

    cos_sim = cosine_similarity(df, df)[0,1]
    similarity.loc[len(similarity.index)] = [prod_name,text, cos_sim]
    

/Users/tanushreedevi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/tanushreedevi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/tanushreedevi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/tanushreedevi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/de

In [9]:
similarity.sort_values(by='similarity_score',ascending=False)[:10]

,product_name,product_review,similarity_score
2356,Yellow,"Pours hazy, caution light yellow in color with...",0.449013
4227,Weissbier Hell\nKlosterbrauerei Andechs,"Pours hazy, gold in color with one inch head. ...",0.426401
3678,Strata Points,Pours a very cloudy opaque golden color with a...,0.410997
5274,Blackout Stout\nGreat Lakes Brewing Co.,Pours a jet black with normal carbonation. hea...,0.408248
2255,Stick Season,"Had on tap. Pours ultra hazy, gold in color wi...",0.400000
2503,Big Mango,"Pours hazy, orange gold in color with one quar...",0.400000
3432,New Day,"Pours ultra hazy, pale gold in color with one ...",0.392232
2286,King Jammy,"Pours ultra hazy, orange in color with three q...",0.390434
3681,Curiosity One Hundred Twenty Two,"Pours hazy, orange gold in color with one quar...",0.384900
3405,Electric Skies,"Had on tap. Pours ultra hazy, caution light ye...",0.384900


### TASK D - Sentiment Analysis

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    final_score = sentiment_dict['compound']
    return final_score

In [11]:
sentiment = []

for i in range(len(text_file)):
    prod_name = comments['product_name'][i]
    text = str(text_file[i])
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = str(no_punct) + str(char)
            
    tokens = [t for t in no_punct.split()]
    filtered_sentence = [w.lower() for w in tokens if not w.lower() in stop_words]
    filtered_sentence_fnl = " ".join(filtered_sentence)
    sentiment.append(sentiment_scores(filtered_sentence_fnl))

In [12]:
score_df = pd.DataFrame(similarity)
score_df['sentiment_score'] = sentiment
score_df[:10]

,product_name,product_review,similarity_score,sentiment_score
0,Lou Pepe - Framboise,Enjoyed this in Brussels. Love raspberry and t...,0.000000,0.9538
1,Lou Pepe - Framboise,"750 ml corked and capped bottle, courtesy of C...",0.077152,0.7579
2,Lou Pepe - Framboise,2015 vintage. Tart raspberries with typical fu...,0.000000,0.5423
3,Lou Pepe - Framboise,Today our fence is being repaired after a Torn...,0.175562,0.9042
4,Lou Pepe - Framboise,"750ml bottle at share. Ruby colored pour, head...",0.208514,0.8093
5,Lou Pepe - Framboise,Beautiful red slightly pinkish color with a th...,0.119523,0.8481
6,Lou Pepe - Framboise,Vibrant ruby with a soft head settling to a pe...,0.124035,0.5574
7,Appervation,"#78, yummy!",0.000000,0.5267
8,Appervation,Straight chilling. Nothing crazy like Chief br...,0.000000,0.8698
9,Appervation,"12.7 oz bottle, vintage 2020, poured into an U...",0.147442,0.8271


### Task E - Evaluation Score using Multiplicative Model 

In [13]:
score_df['evaluation_score'] = score_df['sentiment_score'] * score_df['similarity_score']
score_df[:10]

,product_name,product_review,similarity_score,sentiment_score,evaluation_score
0,Lou Pepe - Framboise,Enjoyed this in Brussels. Love raspberry and t...,0.000000,0.9538,0.000000
1,Lou Pepe - Framboise,"750 ml corked and capped bottle, courtesy of C...",0.077152,0.7579,0.058473
2,Lou Pepe - Framboise,2015 vintage. Tart raspberries with typical fu...,0.000000,0.5423,0.000000
3,Lou Pepe - Framboise,Today our fence is being repaired after a Torn...,0.175562,0.9042,0.158743
4,Lou Pepe - Framboise,"750ml bottle at share. Ruby colored pour, head...",0.208514,0.8093,0.168751
5,Lou Pepe - Framboise,Beautiful red slightly pinkish color with a th...,0.119523,0.8481,0.101367
6,Lou Pepe - Framboise,Vibrant ruby with a soft head settling to a pe...,0.124035,0.5574,0.069137
7,Appervation,"#78, yummy!",0.000000,0.5267,0.000000
8,Appervation,Straight chilling. Nothing crazy like Chief br...,0.000000,0.8698,0.000000
9,Appervation,"12.7 oz bottle, vintage 2020, poured into an U...",0.147442,0.8271,0.121949


In [14]:
recommendation_df = pd.DataFrame(score_df.groupby('product_name')['evaluation_score'].mean())
recommendation_df.sort_values(by='evaluation_score',ascending=False)

,evaluation_score
product_name,
Big Passion Fruit,0.260925
Double Shot - Barrel Blend,0.198351
Holographic,0.182528
Juicy All The Way!,0.177848
Sea Floor,0.172439
...,...
The Adjunct Trail - Bourbon Barrel-Aged,0.000000
Chemtrailmix - Rye Barrel,0.000000
Double Nelson,-0.002749


#### Based on the evaluation score, Big Passion Fruit, Double Shot - Barrel Blend and Holohraphic beers are positively discussed in terms of head, taste, carbonation & pours 

### Task F - SpaCy Score

In [15]:
#code to calculate spaCy and BoW similarity scores from two texts
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

#spaCy uses word vectors for medium (md) and large (lg)
nlp = spacy.load('en_core_web_md') 
spaCy_sim = []

for i in range(len(text_file)):
    prod_name = comments['product_name'][i]
    text = str(text_file[i])
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = str(no_punct) + str(char)
            
    tokens = [t for t in no_punct.split()]
    filtered_sentence = [w.lower() for w in tokens if not w.lower() in stop_words]
    filtered_sentence_fnl = " ".join(filtered_sentence)
    
    doc1 = nlp(filtered_sentence_fnl)
    doc2 = nlp(list_attributes_text)
    spaCy_sim.append(doc1.similarity(doc2))

/var/folders/_j/hp54sf295b9bd0bczykcmwvm0000gn/T/ipykernel_27256/3233427730.py:25: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  spaCy_sim.append(doc1.similarity(doc2))


In [16]:
score_df['spacy_score'] = spaCy_sim
score_df.loc[:10]

,product_name,product_review,similarity_score,sentiment_score,evaluation_score,spacy_score
0,Lou Pepe - Framboise,Enjoyed this in Brussels. Love raspberry and t...,0.000000,0.9538,0.000000,0.681083
1,Lou Pepe - Framboise,"750 ml corked and capped bottle, courtesy of C...",0.077152,0.7579,0.058473,0.655162
2,Lou Pepe - Framboise,2015 vintage. Tart raspberries with typical fu...,0.000000,0.5423,0.000000,0.564445
3,Lou Pepe - Framboise,Today our fence is being repaired after a Torn...,0.175562,0.9042,0.158743,0.686059
4,Lou Pepe - Framboise,"750ml bottle at share. Ruby colored pour, head...",0.208514,0.8093,0.168751,0.755515
5,Lou Pepe - Framboise,Beautiful red slightly pinkish color with a th...,0.119523,0.8481,0.101367,0.746200
6,Lou Pepe - Framboise,Vibrant ruby with a soft head settling to a pe...,0.124035,0.5574,0.069137,0.734969
7,Appervation,"#78, yummy!",0.000000,0.5267,0.000000,0.032982
8,Appervation,Straight chilling. Nothing crazy like Chief br...,0.000000,0.8698,0.000000,0.633618
9,Appervation,"12.7 oz bottle, vintage 2020, poured into an U...",0.147442,0.8271,0.121949,0.696596


In [17]:
score_df['evaluation_score_spacy'] = score_df['spacy_score'] * score_df['sentiment_score']
score_df[:10]

,product_name,product_review,similarity_score,sentiment_score,evaluation_score,spacy_score,evaluation_score_spacy
0,Lou Pepe - Framboise,Enjoyed this in Brussels. Love raspberry and t...,0.000000,0.9538,0.000000,0.681083,0.649616
1,Lou Pepe - Framboise,"750 ml corked and capped bottle, courtesy of C...",0.077152,0.7579,0.058473,0.655162,0.496547
2,Lou Pepe - Framboise,2015 vintage. Tart raspberries with typical fu...,0.000000,0.5423,0.000000,0.564445,0.306098
3,Lou Pepe - Framboise,Today our fence is being repaired after a Torn...,0.175562,0.9042,0.158743,0.686059,0.620335
4,Lou Pepe - Framboise,"750ml bottle at share. Ruby colored pour, head...",0.208514,0.8093,0.168751,0.755515,0.611438
5,Lou Pepe - Framboise,Beautiful red slightly pinkish color with a th...,0.119523,0.8481,0.101367,0.746200,0.632852
6,Lou Pepe - Framboise,Vibrant ruby with a soft head settling to a pe...,0.124035,0.5574,0.069137,0.734969,0.409672
7,Appervation,"#78, yummy!",0.000000,0.5267,0.000000,0.032982,0.017371
8,Appervation,Straight chilling. Nothing crazy like Chief br...,0.000000,0.8698,0.000000,0.633618,0.551121
9,Appervation,"12.7 oz bottle, vintage 2020, poured into an U...",0.147442,0.8271,0.121949,0.696596,0.576155


In [18]:
recommendation_df_spacy = pd.DataFrame(score_df.groupby('product_name')['evaluation_score_spacy'].mean())
recommendation_df_spacy.sort_values(by='evaluation_score_spacy',ascending=False)

,evaluation_score_spacy
product_name,
8th Anniversary,0.738706
DBL Lux,0.725130
Triple Shot - House Blend,0.719423
Quadruple Shot - Monochrome,0.719415
Red Eye November,0.718869
...,...
Project Find The Limit #5,0.119154
Glow Sticks,0.096031
Project Find The Limit #2,0.029719


#### 8th Anniversary, DBL Lux & Friday (2022) are top recommendations based on Spacy scores & sentiment

In [19]:
pct_spacy = len(score_df[score_df['spacy_score']>0])/len(score_df)
pct_cosine = len(score_df[score_df['similarity_score']>0])/len(score_df)

print('% of reviews with attributes - based on cosine similarity: ',round(pct_cosine*100,2), '%')
print('% of reviews with attributes - based on spacy similarity: ',round(pct_spacy*100,2), '%')

% of reviews with attributes - based on cosine similarity:  70.29 %
% of reviews with attributes - based on spacy similarity:  99.5 %


#### Spacy seeks to understand the context of what's being said as opposed to taking it at face value of word usage. Possibly the reason for this huge difference in % of reviews associated with the attributes

### Task G

In [20]:
comments_agg = pd.DataFrame(comments.groupby('product_name')['user_rating'].mean())
comments_agg.sort_values(by='user_rating',ascending=False)[:3]

,user_rating
product_name,
The Adjunct Trail - Bourbon Barrel-Aged,4.9300
Beer:Barrel:Time (Rye 2022),4.8625
Triple Shot - House Blend,4.8400


##### When taking the average ratings we see that the top three beers are:

The Adjunct Trail - Bourbon Barrel-Aged 4.930000 <br>
Beer:Barrel:Time (Rye 2022)	 4.8625
Triple Shot - House Blend	4.8400 <br>

##### When looking at the evaluation scores the top 3 we are seeing are:

Big Passion Fruit 0.260925 <br>
Double Shot - Barrell Blend 0.198351 <br>
Holographic 0.182528 <br>

The recommendations are not the same when considering the average ratings vs the evaluation score (similarity score + sentiment score). They are totally different because the senitment analysis is taking into consideration the NLP. This way is organizing based off opinions through selecting tone and emotions of words on a product while the user rating is only considered based off the user's overall thoughts. 

However, since the sentiment analysis has emphasis on attributes of the product it may not actually be contributing to the ratings overall. The user rating in this case might be a better way to determine what product has an overall better flavor and popularity.

In summary, sentiment + similarity analysis are better ways to offer recommendations since they take into account considerations of the user unlike the ratings system which is more of a commentary on product as a whole

In [26]:
def rem_punc(s):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in s:
        if char not in punctuations:
            no_punct = str(no_punct) + str(char)
    
    return no_punct

In [37]:
def tkn(no_punct):
    tokens = [t.lower() for t in no_punct.split()]
    filtered_sentence = [w.lower() for w in tokens if not w.lower() in stop_words]

    return filtered_sentence

In [83]:
import numpy as np 

joint_prob_dict = {}
lift_dict = {}
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
lst = []
attribute_words = ['head','taste','carbonation','pours']


for m,k in enumerate(attribute_words):
        samp_size = len(text_file)
        lst.append(k)
        for n,j in enumerate(['Twice the Daily Serving: Raspberry','Beer:Barrel:Time (Rye 2022)',
                              'Double Shot - Barrel Blend','The Adjunct Trail - Bourbon Barrel-Aged',
                              'Lou Pepe - Framboise','Aurelian Lure','Appervation',
                              'Big Mango','Big Passion Fruit','Holographic']):
            cnt=0
            if j!=k:
                for i in range(len(text_file)):
                    text = str(text_file[i]) + ' ' + str(comments['product_name'][i])
                    no_punct = rem_punc(text)
                    filtered_sentence_final_lst = tkn(no_punct) 
                    
                    if k in filtered_sentence_final_lst and j == comments['product_name'][i]:
                        cnt = cnt + 1
                    else:
                        cnt = cnt + 0
        
                joint_prob_dict[(k,j)] = cnt
                cnt_k = int(freq_nostop.loc[freq_nostop['word']==k,'count'])
                cnt_j = int(brand_agg.loc[brand_agg['product_name']==j,'product_review'])
                lift_dict[(k,j)] = (joint_prob_dict[(k,j)]*samp_size)/(cnt_k * cnt_j)

In [84]:
lift_interim_aspire = pd.DataFrame.from_dict(lift_dict, orient='index').reset_index()
lift_interim_aspire.columns = ['combination','lift']
lift_interim_aspire[['attribute','brand_2']] = pd.DataFrame(lift_interim_aspire['combination'].tolist(), index=lift_interim_aspire.index)
lift_final_aspire = lift_interim_aspire[['attribute','brand_2','lift']].sort_values(by='attribute',ascending=True)
lift_pvt_aspire = lift_final_aspire.pivot(index='brand_2', columns='attribute', values='lift')
lift_pvt_aspire['Average Lift'] = lift_pvt_aspire.mean(axis=1)
lift_pvt_aspire.sort_values(by='Average Lift',ascending=False)

attribute,carbonation,head,pours,taste,Average Lift
brand_2,,,,,
Big Passion Fruit,3.488296,1.811193,3.513158,2.374213,2.796715
Holographic,3.488296,1.811193,0.878289,2.374213,2.137998
Double Shot - Barrel Blend,0.000000,1.811193,3.513158,2.374213,1.924641
Aurelian Lure,3.488296,1.811193,0.000000,2.374213,1.918425
Big Mango,2.325531,1.207462,2.342105,0.791404,1.666626
Beer:Barrel:Time (Rye 2022),0.000000,0.452798,0.878289,1.780660,0.777937
Appervation,0.996656,0.517484,1.003759,0.339173,0.714268
Lou Pepe - Framboise,0.498328,1.293709,0.000000,1.017520,0.702389
Twice the Daily Serving: Raspberry,0.000000,0.000000,1.171053,1.187106,0.589540


##### To carry out the lift analysis, we considered the top 4 attributes & tried to understand how often they were co-mentioned for the list of beers we chose. 

##### It can be seen from this that Big Passion Fruit, Double-Shot Barrel Blend & Holographic have the highest overall lift which is in keeping with our cosine similarity analysis recommendations. Thus, it's fair to say that the cosine similarity & lift work similarly in this specific use case. 

##### In terms of recommendations based on lift values, Big Passion Fruit & Holographic seem the most similar except that Big Passion Fruit is better known for its pours. 

##### It's also interesting that The Adjunct Trail which had a high user rating doesn't comply well with any of these attributes which goes to show that similarity scores are better suited for recommendations